# Local Linear Iterative Stockholder Analysis (L-ISA) schemes

## Non-linear optimization problem

### Convex optimization method


In [ ]:
from setup import prepare_argument_dict, prepare_grid_and_dens, print_results

from horton_part import LinearISAWPart

mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")


def main_lisa():
    """Local LISA by solving convex optimization problem."""
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 101
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa()

### Trust-region method

In [ ]:
def main_lisa_301():
    """Local LISA by solving convex optimization problem."""
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 301
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa_301()

One can also add constraint explicitly

In [ ]:
def main_lisa_302():
    """Local LISA by solving convex optimization problem."""
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 302
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa_302()

## Non-linear equations (fixed-point equations)

### Self-consistent method


In [ ]:
def main_lisa_201():
    """Self-consistent solver."""
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 201
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa_201()

### DIIS

In [ ]:
def main_lisa_202():
    """Self-consistent solver."""
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 202
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa_202()

### Newton method

In [ ]:
def main_lisa_203():
    """Self-consistent solver."""
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 203
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)


main_lisa_203()